# Obtener oferta academica

In [1]:
# Libraries for selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
# Waited conditionals for Selenium
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Data Manipulations libraries
import pandas as pd

# Regular expressions library
import re

# Datetime library
import time

- Lanzar el navegador en la página del SIGMAA

In [2]:
# Open the web browser
driver = webdriver.Edge()
# Maximize the window
driver.maximize_window()

- Accede exitosamente al formulario de ingreso

In [3]:
# Acceder al SIGMAA
url = 'https://uclb.ucaribe.edu.mx/sigmaav2/'
driver.get(url)

- Iniciar sesión con las credenciales del estudiante

In [4]:
# Biblioteca para leer archivos
import os
# Biblioteca para leer .env
from dotenv import load_dotenv
# Cargamos variables de ambiente
load_dotenv()

# Obtener las credenciales del estudiante
id_user = os.getenv('ID')
password = os.getenv('PASS')
MYSQL_ADDON_URI = os.getenv('MYSQL_ADDON_URI')

# Plan de estudios del estudiante
study_plan = "2016ID"

In [5]:
# Buscar el campo de usuario y escribir el username
userinput = driver.find_element(By.XPATH, '/html/body/div[2]/form/div/span[2]/input')
userinput.send_keys(id_user)

In [6]:
# Buscar el campo de contraseña y escribir el password
passinput = driver.find_element(By.XPATH, '/html/body/div[2]/form/div/input')
passinput.send_keys(password)

In [7]:
# Buscar el botón de submit y dar clic para iniciar sesión
submitinput = driver.find_element(By.XPATH, '/html/body/div[2]/form/button')
submitinput.click()

In [8]:
# Ir a la sección de las tablas de la oferta académica
ofertaAcademica = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/div/div/ul[2]/li[1]/a')
ofertaAcademica.click()

- Establecer conexión con la base de datos

In [9]:
# Importar biblioteca para conexion a base de datos
from sqlalchemy import create_engine
# Establecer conexion a base de datos
engine = create_engine(MYSQL_ADDON_URI)

- Extraer tablas

In [10]:
# xpath de pestañas
pestañas = [
    # Secciones
    '/html/body/center/ul/li[1]/a',
    # Talleres
    '/html/body/center/ul/li[2]/a',
    # Lengua Extranjera
    '/html/body/center/ul/li[3]/a'
]

In [11]:
for index, p in enumerate(pestañas):
    # Cambiamos a cada pestaña
    pestaña = driver.find_element(By.XPATH, p)
    pestaña.click() 

    # Extraer el contenido html de la pagina actual en selenium
    dfs = pd.read_html(driver.page_source)

    if index == 0:
        # Nombres para las columnas
        columnas = ['Tipo', 'Clave', 'Seccion', 'Asignatura', 'Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'U1', 'U2', 'U3', 'U4']
    else:
        # Nombres para las columnas
        columnas = ['Clave', 'Seccion', 'Asignatura', 'Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'U1', 'U2', 'U3', 'U4', 'U5']
        
    # Columnas que se desean conservar
    columnas_deseadas = [ 'Clave', 'Seccion', 'Asignatura', 'Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado']

    # Recorre cada tabla en la vista actual 
    for i in range(1, len(dfs)):
        # Renombra las columnas
        dfs[i] = dfs[i].set_axis(columnas, axis=1)
        # Mantiene las columnas
        dfs[i] = dfs[i].loc[:, dfs[i].columns.isin(columnas_deseadas)]

    for j in range(1, len(dfs)):
        mydf = dfs[j]

        splits = []
        string = mydf['Asignatura']

        for i in range(len(string)):
            lista = string[i].split('  ')
            splits.append(lista)

        asignaturas = []
        profesores = []
        modalidades = []

        for i in splits:
            asignaturas.append(i[0])
            profesores.append(i[1])
            modalidades.append(i[2])

        mydf['Asignatura'] = asignaturas
        mydf.insert(3, 'Profesor', profesores)
        mydf.insert(4, 'Modalidad', modalidades)

        if index == 0:
            nombre_tabla = 'secciones_' + study_plan
        elif index == 1:
            nombre_tabla = 'talleres_' + study_plan
        else:
            nombre_tabla = 'lengua_extranjera_' + study_plan
            
        if j == 1:
            dfs[j].to_sql(nombre_tabla, con = engine, if_exists='replace')
        else:
            dfs[j].to_sql(nombre_tabla, con = engine, if_exists='append')

- Cerrar la sesión del usuario en SIGMAA

In [12]:
# Cerrar sesión
logout = driver.find_element(By.XPATH, '/html/body/div[2]/div/form/a')
logout.click()

In [13]:
# Cerrar el navegador
driver.quit()